# Tests

In [26]:
import numpy as np
from Classes.PercepTeacher import PerceptronTeacher
from Functions.function_of_activation import sigmoid, dsigmoid
from Functions.errorFunctions import MSE
from Classes.Layer import Layer
from Classes.LayerGen import LayerGenerator
from random import random
from Classes.Perceptron import Perceptron


In [27]:
def batch_spliter(x, y, size, num):
    batches = list()
    for _ in range(num):
        js = np.random.shuffle(np.arange(size))
        batches.append((x[js][0], y[js][0]))
    return batches


In [28]:
weights = [np.random.rand(i, j) for i, j in zip([3, 2, 1], [3, 3, 2])]
for w in weights:
    print(w, w.shape)


[[0.54953911 0.42420275 0.52401223]
 [0.43247065 0.09968135 0.9483311 ]
 [0.72150446 0.29385752 0.63674547]] (3, 3)
[[0.03657781 0.23667263 0.74096574]
 [0.11598799 0.8650552  0.25264241]] (2, 3)
[[0.12113721 0.20623696]] (1, 2)


In [29]:
test_layer = Layer(weights=weights[0], 
                   dias=random(),
                   summator=sum,
                   dsummator=lambda a: 1,
                   activation_function=sigmoid,
                   dactivation_function=dsigmoid)

In [30]:
test_x = np.random.rand(1, 3)
print(test_x)
print(test_layer.get_output(test_x))


[[0.09369533 0.45236675 0.51519608]]
[0.74262814 0.75408319 0.74552796]


In [31]:
test_layer_mass = LayerGenerator(sizes_of_input_layer=3, 
                                 sizes_of_hidden_layers=[4, 3, 2], 
                                 sizes_of_output_layer=1,
                                 summators=[sum, sum, sum, sum, sum],
                                 dsummators=[lambda a: 1,
                                             lambda a: 1, 
                                             lambda a: 1,
                                             lambda a: 1,
                                             lambda a: 1],
                                 activation_functions=[sigmoid, 
                                                       sigmoid, 
                                                       sigmoid, 
                                                       sigmoid,
                                                       sigmoid],
                                 dactivation_functions=[dsigmoid, 
                                                        dsigmoid, 
                                                        dsigmoid, 
                                                        dsigmoid,
                                                        dsigmoid],
                                 function_of_generation=lambda a,
                                 b: np.random.rand(a, b)).generate()
test_layer_mass


In [32]:
for l in test_layer_mass:
    print(l.get_weights(), l.get_weights().shape)


[[0.00689423 0.32084301 0.47704842]
 [0.37208531 0.29350152 0.97811372]
 [0.0204951  0.16138462 0.78888435]
 [0.8301563  0.84271107 0.74661011]] (4, 3)
[[0.37048328 0.78234334 0.2019238  0.67929688]
 [0.06486695 0.74002785 0.93165677 0.20551772]
 [0.24543395 0.89274848 0.06870737 0.20094148]] (3, 4)
[[0.27081289 0.4651133  0.9704673 ]
 [0.10982162 0.36935121 0.99100978]] (2, 3)
[[0.07571588 0.94890836]] (1, 2)


In [33]:
test_teacher = PerceptronTeacher(layers=test_layer_mass,
                                 step=100,
                                 batch_spliter=lambda a, b: batch_spliter(a, b, a.shape[0]//2, 1),
                                 epochs=100,
                                 error_function=MSE,
                                 derror_function=lambda x, y: 2 * x - 2 * y)


In [34]:
test_X = np.array([[1, 2, 3], [2, 3, 4], [4, 5, 6],
                   [7, 8, 1], [9, 8, 1], [7, 2, 1], [5, 8, 1], [1, 8, 2], [7, 3, 1], [7, 7, 7], [2, 2, 1], [1, 1, 7], [4, 3, 2], [7, 8, 1]])
test_y = np.sum(test_X, axis=1).reshape(
    test_X.shape[0], 1).reshape(test_X.shape[0], 1)

a = 0
b = 1
x_min_max = a + (test_X - test_X.min())/(test_X.max() - test_X.min()) * (b-a)
y_min_max = a + (test_y - test_y.min())/(test_y.max() - test_y.min()) * (b-a)


In [35]:
x_min_max


array([[0.   , 0.125, 0.25 ],
       [0.125, 0.25 , 0.375],
       [0.375, 0.5  , 0.625],
       [0.75 , 0.875, 0.   ],
       [1.   , 0.875, 0.   ],
       [0.75 , 0.125, 0.   ],
       [0.5  , 0.875, 0.   ],
       [0.   , 0.875, 0.125],
       [0.75 , 0.25 , 0.   ],
       [0.75 , 0.75 , 0.75 ],
       [0.125, 0.125, 0.   ],
       [0.   , 0.   , 0.75 ],
       [0.375, 0.25 , 0.125],
       [0.75 , 0.875, 0.   ]])

In [36]:
y_min_max


array([[0.0625],
       [0.25  ],
       [0.625 ],
       [0.6875],
       [0.8125],
       [0.3125],
       [0.5625],
       [0.375 ],
       [0.375 ],
       [1.    ],
       [0.    ],
       [0.25  ],
       [0.25  ],
       [0.6875]])

In [37]:
test_teacher.teach(x_min_max, y_min_max)


In [38]:
bl = test_teacher.teach(x_min_max, y_min_max)


In [39]:
a = Perceptron(bl)
res = [a.predict(i) for i in x_min_max]
print(MSE(res, y_min_max))
for p, y in zip(res, y_min_max):
    print(f'p:{p},\t y:{y},\t d:{abs(p-y)}')

0.07735513247723022
p:[0.57879177],	 y:[0.0625],	 d:[0.51629177]
p:[0.58430038],	 y:[0.25],	 d:[0.33430038]
p:[0.58430038],	 y:[0.625],	 d:[0.04069962]
p:[0.58430038],	 y:[0.6875],	 d:[0.10319962]
p:[0.58430038],	 y:[0.8125],	 d:[0.22819962]
p:[6.50996924e-30],	 y:[0.3125],	 d:[0.3125]
p:[0.58430038],	 y:[0.5625],	 d:[0.02180038]
p:[0.58430038],	 y:[0.375],	 d:[0.20930038]
p:[6.50996997e-30],	 y:[0.375],	 d:[0.375]
p:[0.58430038],	 y:[1.],	 d:[0.41569962]
p:[6.50996924e-30],	 y:[0.],	 d:[6.50996924e-30]
p:[0.58430038],	 y:[0.25],	 d:[0.33430038]
p:[4.76354074e-26],	 y:[0.25],	 d:[0.25]
p:[0.58430038],	 y:[0.6875],	 d:[0.10319962]


# Laba

In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from Functions.errorFunctions import MAE, MAPE, MSE, RMSE, R2
from sklearn.metrics import r2_score


In [41]:
models = pd.read_csv('../Data/DataLaba1/energy_task_moded.csv')
models = models.drop('year', axis=1)
a = 0
b = 1
# for i in models.columns:
#     models[i] = (models[i]-models[i].mean())/models[i].std()
#     models[i] = a + (models[i] - models[i].min()) / (models[i].max() - models[i].min()) * (b-a)


In [42]:
X_train, X_test, y_train, y_test = train_test_split(models.drop('Appliances', axis=1), 
                                                    a + (models['Appliances'] - models['Appliances'].min()) / (models['Appliances'].max() - models['Appliances'].min()) * (b-a), 
                                                    test_size=0.6)


In [43]:
X_train.shape

(7894, 28)

In [44]:
layer_mass = LayerGenerator(sizes_of_input_layer=X_train.shape[1],
                                 sizes_of_hidden_layers=[
                                     X_train.shape[1]//2, X_train.shape[1]//4, X_train.shape[1]//8],
                                 sizes_of_output_layer=1,
                                 summators=[sum, sum, sum, sum, sum],
                                 dsummators=[lambda a: 1,
                                             lambda a: 1,
                                             lambda a: 1,
                                             lambda a: 1,
                                             lambda a: 1],
                                 activation_functions=[sigmoid,
                                                       sigmoid,
                                                       sigmoid,
                                                       sigmoid,
                                                       sigmoid],
                                 dactivation_functions=[dsigmoid,
                                                        dsigmoid,
                                                        dsigmoid,
                                                        dsigmoid,
                                                        dsigmoid],
                                 function_of_generation=lambda a,
                                 b: np.random.rand(a, b)).generate()
test_layer_mass


In [45]:
for i in layer_mass:
    print(i.get_weights())


[[0.76895278 0.4927416  0.80613165 0.72270407 0.58429893 0.45695687
  0.24710244 0.911156   0.27367069 0.19021269 0.88851604 0.11416114
  0.04602529 0.20746854 0.21924714 0.3058643  0.92209666 0.21482334
  0.72261004 0.81493434 0.62583968 0.51076173 0.08745803 0.05328135
  0.00629017 0.41890377 0.92020353 0.78852917]
 [0.58400805 0.98286035 0.48973658 0.17330512 0.94029152 0.93362251
  0.8079149  0.14563192 0.73452383 0.75779767 0.21033014 0.75566334
  0.7183569  0.70986995 0.25617645 0.5314207  0.8106289  0.14739964
  0.07811451 0.55788649 0.40105865 0.97475614 0.42192211 0.54722536
  0.07343893 0.6315441  0.41930767 0.57768002]
 [0.10734296 0.37804621 0.63429574 0.31361463 0.20333786 0.81228469
  0.90877759 0.74972709 0.16611331 0.8193069  0.9534692  0.71030882
  0.08644817 0.80142309 0.29705673 0.11567607 0.39471018 0.21397647
  0.02569245 0.14403768 0.00578243 0.80912781 0.00109192 0.67945374
  0.44063551 0.80728473 0.23682979 0.26711538]
 [0.07639381 0.55012816 0.80448552 0.284911

In [46]:
teacher = PerceptronTeacher(layers=layer_mass,
                                 step=1,
                                 batch_spliter=lambda a, b: batch_spliter(
                                     a, b, a.shape[0]//2, 1),
                                 epochs=100,
                                 error_function=MSE,
                                 derror_function=lambda x, y: 2 * x - 2 * y)


In [47]:
tl = teacher.teach(X_train.to_numpy(), y_train.to_numpy())


c:\Users\Mike\Desktop\4semak\Maching_learning\MyMethods\Functions\function_of_activation.py:5: RuntimeWarning: overflow encountered in double_scalars
  return 1 / (1 + np.e**(-x))


In [ ]:
a = Perceptron(tl)
res = [a.predict(i)[0] for i in X_test.to_numpy()]
print(MSE(res, y_test.to_numpy()))
for p, y in zip(res, y_test.to_numpy()):
    print(f'p:{p},\t y:{y},\t d:{abs(p-y)}')


0.009123460068539186
p:0.08248486240632306,	 y:0.056074766355140186,	 d:0.02641009605118288
p:0.08248486240632306,	 y:0.04672897196261682,	 d:0.035755890443706244
p:0.08248486240632306,	 y:0.009345794392523364,	 d:0.0731390680137997
p:0.08248486240632306,	 y:0.056074766355140186,	 d:0.02641009605118288
p:0.08248486240632306,	 y:0.08411214953271028,	 d:0.0016272871263872113
p:0.08248486240632306,	 y:0.09345794392523364,	 d:0.010973081518910577
p:0.08248486240632306,	 y:0.028037383177570093,	 d:0.05444747922875297
p:0.08248486240632306,	 y:0.028037383177570093,	 d:0.05444747922875297
p:0.08248486240632306,	 y:0.3364485981308411,	 d:0.253963735724518
p:0.08248486240632306,	 y:0.12149532710280374,	 d:0.03901046469648067
p:0.08248486240632306,	 y:0.06542056074766354,	 d:0.01706430165865952
p:0.08248486240632306,	 y:0.037383177570093455,	 d:0.04510168483622961
p:0.08248486240632306,	 y:0.056074766355140186,	 d:0.02641009605118288
p:0.08248486240632306,	 y:0.3644859813084112,	 d:0.28200111890

In [ ]:
r2_score(res, y_test)
# -1.183786126545848e+31, 0.01, 100


-4.737171305284396e+31

In [ ]:
for i in tl:
    print(i.get_weights())

[[ 7.89146207e-01 -2.66961395e+00 -4.66698501e+00 -2.07191200e+00
  -5.15848610e+00 -2.12548237e+00 -4.54799266e+00 -1.66379433e+00
  -4.67555120e+00 -1.72219742e+00 -5.37115839e+00 -5.55489231e-01
  -7.41461575e+00 -1.79672775e+00 -3.30494653e+00 -1.97662584e+00
  -4.85378373e+00 -1.96143605e+00 -5.07819495e+00  2.27652553e-01
  -9.31287310e+01 -9.75548086e+00 -7.47640052e-02 -4.11153305e+00
  -4.13907558e-01 -2.81233446e+00 -2.70931721e-01 -1.99661884e+01]
 [ 7.85044466e-01 -2.16333066e+00 -3.92503746e+00 -1.44719420e+00
  -4.91010746e+00 -2.58137534e+00 -4.58166041e+00 -1.60795956e+00
  -4.81088622e+00 -2.25268264e+00 -5.64762642e+00  2.44175465e-01
  -7.07881233e+00 -1.88541934e+00 -3.56464043e+00 -2.10212002e+00
  -5.28375109e+00 -1.61823222e+00 -4.76720163e+00 -3.95560793e-01
  -9.31710680e+01 -1.00747008e+01 -9.41729146e-01 -4.76042954e+00
   1.84145343e-01 -3.36435764e+00  1.08678616e-01 -1.91266837e+01]
 [ 6.19885857e-01 -2.11819671e+00 -4.34449424e+00 -1.52013295e+00
  -5.191